# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_data = pd.read_csv("weather_data.csv")
vacation_data.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,hilo,19.73,-155.09,73.00,88,90,9.17,US,1607624576
1,thompson,55.74,-97.86,6.80,71,20,8.05,CA,1607624670
2,ushuaia,-54.80,-68.30,62.60,36,40,13.87,AR,1607624811
3,butaritari,3.07,172.79,82.02,74,81,7.27,KI,1607624811
4,bredasdorp,-34.53,20.04,66.20,72,46,5.82,ZA,1607624811
5,hobart,-42.88,147.33,48.20,66,90,9.17,AU,1607624664
6,bilma,18.69,12.92,78.78,17,37,7.81,NE,1607624811
7,pilar,-9.60,-35.96,86.00,62,75,17.22,BR,1607624649
8,punta arenas,-53.15,-70.92,59.00,66,75,9.17,CL,1607624736
9,fortuna,40.60,-124.16,50.00,98,19,1.01,US,1607624812


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

locations = vacation_data[["Lat", "Lng"]].astype(float)
humidity = vacation_data["Humidity"].astype(float)
center = (41.37,-73.41)
fig = gmaps.figure(zoom_level=2, center=center)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#filter for ideal weather criteria
new_vacation = vacation_data.loc[(vacation_data["Max Temp"] < 80) & (vacation_data["Max Temp"] > 70) & (vacation_data["Wind Speed"] < 10) 
                                 & (vacation_data["Cloudiness"] == 0)] 
new_vacation.dropna().head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
46,panaba,21.28,-88.27,75.25,60,0,6.82,MX,1607624820
104,noumea,-22.28,166.46,75.20,83,0,3.36,NC,1607624839
242,tombouctou,20.00,-3.00,70.36,19,0,9.15,ML,1607624871
292,agadez,19.75,10.25,74.16,19,0,6.02,NE,1607624888
426,china,25.70,-99.23,74.73,35,0,7.16,MX,1607624860
448,thap than,15.46,99.89,73.00,76,0,4.68,TH,1607624917


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
#display 10 records or less with new columns - City, Country, Lat, Lng
hotel_df = new_vacation.loc[:,["City","Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Lat,Lng,Hotel Name
46,panaba,MX,21.28,-88.27,
104,noumea,NC,-22.28,166.46,
242,tombouctou,ML,20.00,-3.00,
292,agadez,NE,19.75,10.25,
426,china,MX,25.70,-99.23,
448,thap than,TH,15.46,99.89,


In [6]:
#build query
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"keyword": "hotel", "type": "hotel", "radius": 5000, "key": g_key}

#iterate through hotel dataframe and append hotel name to the result
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    params["location"] = f'{lat}, {lng}'
    
    #assemble URL and API request
    print(f'Retrieving results for {index}: {city_name}.')
    response = requests.get(url, params=params).json()
    
    results = response["results"]
    
    try:
        print(f'The closest hotel in {city_name} is {results[0]["name"]}.')
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
          
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")   
    
    time.sleep(1)
          
print("----------------------------")                               

Retrieving results for 46: panaba.
Missing field/result... skipping.
Retrieving results for 104: noumea.
The closest hotel in noumea is GONDWANA HOTEL.
Retrieving results for 242: tombouctou.
Missing field/result... skipping.
Retrieving results for 292: agadez.
Missing field/result... skipping.
Retrieving results for 426: china.
The closest hotel in china is Hotel Hacienda el Cuchillo Inn.
Retrieving results for 448: thap than.
The closest hotel in thap than is BN​ Resort.
----------------------------


In [7]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
46,panaba,MX,21.28,-88.27,
104,noumea,NC,-22.28,166.46,GONDWANA HOTEL
242,tombouctou,ML,20.00,-3.00,
292,agadez,NE,19.75,10.25,
426,china,MX,25.70,-99.23,Hotel Hacienda el Cuchillo Inn
448,thap than,TH,15.46,99.89,BN​ Resort


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer on top of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))